In [ ]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np
from utils import run_pll, delete_all_files_in_folder
from run_pll import run_dr_raw, run_autognet_raw, run_dr_em_raw

def prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree):
    # 1. Simulate or load data
    adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
    print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

    tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
    rho = np.array([[0,0.1,0.2],
                    [0.1,0,0.1],
                    [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
    nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
    gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
    beta = np.array([-1*min_degree,2*min_degree,-2,2,0.1,-2,0.1,2,0.1,0])  # shape (10,)

    Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
        burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

    Y_chain = Y_chain[::5]
    A_chain = A_chain[::5]
    L_chain = L_chain[::5]

    res_truth1 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.7,
        R=100,
        burnin_R=100,
        seed=0
    )
    res_truth2 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.3,
        R=100,
        burnin_R=100,
        seed=0
    )

    ground_truth1 = {}
    ground_truth1['average'] = res_truth1['average']
    ground_truth1['direct'] = res_truth1['direct_effect']
    ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
    ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
    ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
    ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
    ground_truth1['psi_zero'] = res_truth1['psi_zero']

    return adj1, Y_chain, A_chain, L_chain, ground_truth1

def run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec=None):
    # 1. Prepare data
    adj1, Y_chain, A_chain, L_chain, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
    
    treat_p = 0.7
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p,
            'psi_0_gamma_only': False,
            'mispec': mispec} for i in range(len(Y_chain))]

    res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    # save results
    np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)
    delete_all_files_in_folder(f'run/run_dr_raw')


    treat_p = 0.3
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p,
            'psi_0_gamma_only': True,
            'mispec': mispec} for i in range(len(Y_chain))]

    res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    
    # save results
    np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)
    delete_all_files_in_folder(f'run/run_dr_raw')

    ##################################################  RUN DRNET EM #######################################################


    treat_p = 0.7
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p,
            'mispec': mispec} for i in range(len(Y_chain))]

    res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    # save results
    np.save(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)
    delete_all_files_in_folder(f'run/run_dr_em_raw')


    treat_p = 0.3
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p,
            'mispec': mispec} for i in range(len(Y_chain))]

    res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    # save results
    np.save(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)
    delete_all_files_in_folder(f'run/run_dr_em_raw')

    ##################################################  RUN AUTOGNET #######################################################

    if mispec == 'outcome' or mispec is None:
        if mispec == 'outcome':
            np.random.seed(0)
            L_chain = np.random.binomial(n=1, p=0.2, size=L_chain.shape)

        treat_p = 0.7
        args = [{'Y_chain': Y_chain,
                'A_chain': A_chain,
                'L_chain': L_chain,
                'adj': adj1,
                'i': i,
                'treatment_allocation': treat_p} for i in range(len(Y_chain))]

        res_list_dr = run_pll(run_autognet_raw, args, processes=n_cpu)
        res_list_array_dr = np.array(res_list_dr)
        # save results
        np.save(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)
        delete_all_files_in_folder(f'run/run_autog_raw')


        treat_p = 0.3
        args = [{'Y_chain': Y_chain,
                'A_chain': A_chain,
                'L_chain': L_chain,
                'adj': adj1,
                'i': i,
                'treatment_allocation': treat_p} for i in range(len(Y_chain))]

        res_list_dr = run_pll(run_autognet_raw, args, processes=n_cpu)
        res_list_array_dr = np.array(res_list_dr)
        # save results
        np.save(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)
        delete_all_files_in_folder(f'run/run_autog_raw')


In [ ]:
num_sample = 10000
num_burn = 2000
sample_size = 800
n_cpu = 125

min_degree = 1
max_degree = 2

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

1 2


 68%|██████▊   | 8170/12000 [00:43<00:20, 188.75it/s]

In [ ]:
min_degree = 2
max_degree = 5

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

In [ ]:
min_degree = 3
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

In [ ]:
min_degree = 1
max_degree = 4

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

In [ ]:
min_degree = 1
max_degree = 8

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

In [ ]:
min_degree = 1
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

In [ ]:
min_degree = 2
max_degree = 8

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

In [ ]:
min_degree = 2
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

In [ ]:
min_degree = 4
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')